In [2]:
from seeq import spy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import math
import networkx
import time
import gradio
from datetime import datetime

In [3]:
def pi_connect_and_pulldata(df_PITag, startDate, endDate, interval):
    
    """
    example       : pi_connect_and_pulldata(df,'7-Jun-16 06:45:00','7-Jun-16 07:00:00','15s')
    df_PITag      : pandas dataframe containing PI Tag need to pull
    startDate     : start date
    endDate       : end date
    interval      : how frequency
    """
    
    df_return = pd.DataFrame()
    print("Total Tags are ", df_PITag.shape[0])
    
    start_time = time.time()
    print('XXXXXXXXXXXXXX\n')
    #df_pull = spy.pull(df_PITag, start='2020-01-01', end='2020-01-07', grid = '30 min')
    df_pull = spy.pull(df_PITag.iloc[:,:9], start=startDate, end=endDate, grid = interval)    
    df_pull.columns = df_PITag['ShownText']
    
    print(df_pull.head())
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return df_pull

In [4]:
from functools import reduce
import json

def export_tree_json(array_linkage, labels, current_time):
    
    id2name = dict(enumerate(labels))
    T = shc.to_tree(array_linkage , rd=False )

    # Create a nested dictionary from the ClusterNode's returned by SciPy
    def add_node(node, parent ):
        # First create the new node and append it to its parent's children
        newNode = dict( node_id=node.id, children=[] )
        parent["children"].append( newNode )

        # Recursively add the current node's children
        if node.left: add_node( node.left, newNode )
        if node.right: add_node( node.right, newNode )

    # Initialize nested dictionary for d3, then recursively iterate through tree
    #d3Dendro = dict(children=[], name="Root1")
    #add_node( T, d3Dendro )

    # Label each node with the names of each leaf in its subtree
    def label_tree( n ):
        # If the node is a leaf, then we have its name
        if len(n["children"]) == 0:
            leafNames = [ id2name[n["node_id"]] ]

        # If not, flatten all the leaves in the node's subtree
        else:
            leafNames = reduce(lambda ls, c: ls + label_tree(c), n["children"], [])

        # Delete the node id since we don't need it anymore and
        # it makes for cleaner JSON
        del n["node_id"]

        # Labeling convention: "-"-separated leaf names
        #n["name"] = name = "-".join(sorted(map(str, leafNames)))
        n["name"] = name = sorted(leafNames)[-1] 
        #n["name"] = name = sorted(leafNames)[0] 
        return leafNames

    
    # Initialize nested dictionary for d3, then recursively iterate through tree
    d3Dendro = dict(children=[], name="Root1")
    add_node( T, d3Dendro )

    # Create Tree Dendogram
    label_tree( d3Dendro["children"][0] )

    # Output to JSON
    json.dump(d3Dendro, open(current_time+".json", "w"), sort_keys=True, indent=4)

In [5]:
from statsmodels.tsa.stattools import grangercausalitytests

def grangers_causation_topN_list(first_out_tag, data
                                  , test_score_method ='ssr_chi2test'
                                  , decimal_round_digit=9
                                  , maxlag=9
                                  , topN=20): 
    
    #first_out_tag = 'LIT2060_PV'
    #overallTags = ['LIT2060_PV','LIT2061_PV','LIT2062_PV']
    input_df = data
    overall_tags = input_df.columns

    temp_df = pd.DataFrame(np.zeros((1, len(overall_tags))), columns=overall_tags, index=[first_out_tag])
    for y in temp_df.columns:
        #print(y)
        try:
            granger_result = grangercausalitytests(input_df[[first_out_tag, y]], maxlag=maxlag, verbose=False)
            p_values = [round(granger_result[i+1][0][test_score_method][1],decimal_round_digit) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            temp_df.loc[first_out_tag, y] = min_p_value
        except Exception:
            pass
        
    return temp_df.transpose()[first_out_tag].sort_values().head(topN)

In [6]:
#df_PITag=pd.read_csv(r'C:\Users\szjt\Auto_AD\PI_Tag_Reduced.csv', encoding= 'unicode_escape')
def get_tag_data(area, level_YES, main_category):
    df_PITag = pd.read_csv(area+r'_Tags.csv', index_col = 0)
    
    if(level_YES):
        df_PITag = df_PITag[(df_PITag['Level'] == 1)]
    else:
        df_PITag = df_PITag[(df_PITag['AssetTree'].isin(main_category))]
    
    #df_PITag.rename(columns={'Sensor': 'Name'}, inplace=True)
   
    print(df_PITag.head())
    return df_PITag

In [7]:
def convert_inverse_data(input_df, df_PITag_Value):
    
    #Drop rows that have no PI tag data
    input_df.dropna(axis = "columns", how = "all", inplace = True)
    input_df.dropna(axis = 0, how = "all", inplace = True)
    input_df.to_csv("input_df.csv")
    input_df.fillna(method ='ffill', inplace = True)
    #Drop columns that have unique values
    for col in input_df.columns:
        if len(input_df[col].unique()) == 1:
            input_df.drop(col,inplace=True,axis=1)
            print("Column " + col + " has unique values and been dropped.")
    
    df_PITag_inverse = df_PITag_Value[(df_PITag_Value["Valve"]==1)]
    list_tags = df_PITag_inverse['ShownText'].tolist()
    update_df = pd.DataFrame()
    if not list_tags:
        print('Warning: No tag is identified as valve.')
        
    else:
        print('Found some tags are valves')
        print(list_tags)
        for i in list_tags:
            try: #check whether the valve column has been dropped or not
                name = i + '_converted'
                input_df[name] = 100 - input_df[i]   
            except KeyError:
                print("Column "+i+" has been dropped.")
                
     
    return input_df

In [8]:
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import re
#inputData = pd.read_csv(r"C:\Users\tsvd\PIdataJuly2020.csv")

def autoAD_clustering(start_date, end_date, frequency_method, selected_CPP, level_chkbox, tags_main_picker, granger_picker, granger_txt):
    
    print("Get specific Tags from UI")
    df_PITag = get_tag_data(selected_CPP, level_chkbox, tags_main_picker)
    
    print(df_PITag.head())
    print("Get PI Data from Date Range, Frequency and Tags Selections")
    original_df = pi_connect_and_pulldata(df_PITag, start_date, end_date, frequency_method)
    
    pattern = re.compile(r'\D\D-\D.*')
    for col in original_df.columns:
        if re.search(pattern,col):
            original_df.rename(columns={col:re.search(pattern,col).group()},inplace=True)
    
    original_df.to_csv(selected_CPP+"_original_"+datetime.now().strftime("%Y_%m_%d_%H_%M"+".csv"))
    print(original_df.head())
    #original_df = pd.read_csv(r"C:\Users\tsvd\Jo_auto_ad\SACPP_PI_DATA_edited.csv")
    print("Converting data...")
    df_convert_inverse_data = convert_inverse_data(original_df, df_PITag)
    df_convert_inverse_data.to_csv(selected_CPP+"_converted_"+datetime.now().strftime("%Y_%m_%d_%H_%M"+".csv"))
    print("Data Preprocessing...")
     
    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df_convert_inverse_data), columns=df_convert_inverse_data.columns)
    df_scaled = df_scaled.loc[:, (df_scaled != 0).any(axis=0)]
    col_names = df_scaled.columns
    df_scaled = df_scaled.transpose()
        
    #df_scaled.head()
    df_scaled.to_csv(selected_CPP+"_scaled_"+datetime.now().strftime("%Y_%m_%d_%H_%M"+".csv"))
    print("Method Activated!!!")
    print(granger_picker)
    if(granger_picker == "Granger Casuality Test"):
    #if any("Granger Casuality Test" in item for item in granger_picker):
        
        result_msg = grangers_causation_topN_list(granger_txt, df_scaled, decimal_round_digit=9, topN=20, maxlag=6).to_string()
        print("Granger Activated!!!")
        print(result_msg)
        #result_msg = "Granger Completed!!!"
    else :
        
        #fig = plt.figure(figsize=(15, 15))
        #plt.title("Customer Dendograms")

        Z = shc.linkage(df_scaled, method = 'ward', metric = 'euclidean')

        # Plot with Custom leaves
        ax = shc.dendrogram(Z, orientation='left', labels=col_names) #df_scaled.columns)

        id2name = dict(enumerate(col_names))#df_scaled.columns))
        print("Export .json file")
        export_tree_json(Z, col_names, selected_CPP+"_"+datetime.now().strftime("%Y_%m_%d_%H_%M"))
              
        result_msg = "Clustering Completed!!!"
    
    print("!!! AUTO-AD DONE !!!")
    
    return result_msg


In [9]:
import gradio as gr

main_category = ['Coolers', '3 Phase Separators', 'Pumps', 'Glycol Contactors', '2 Phase Separators', 'Stabilizer Reboilers', 
            'Hydrocyclones', 'Filter Coalescers', 'Filters', 'Compressor-Expander Skids', 'WHRUs', 'Glycol Reboilers', 
            'Receivers', 'Launchers', 'Exchangers', 'Heaters', 'Glycol Flashs', 'IGFs', 'Stabilizers', 'Screw Compressors', 
            'Sales Gas Meters', 'Condensate Meters', 'Air Systems', 'Storage Tanks', 'Generator Skids', 'Sales Gas', 
            'Skids_SA-GGC2090-SACPP', 'Skids_SA-GGC2320-SACPP']

start_date = gr.inputs.Textbox(lines=1, default="2020-01-01", label="Start Date (Year-Month-Date Thailand time)")
end_date = gr.inputs.Textbox(lines=1, default="2020-01-07", label="End Date (Year-Month-Date Thailand time)")
frequency_method = gr.inputs.Dropdown(['15 s', '1 min', '15 min', '30 min', '24 h'], label="Sampling frequency")
selected_CPP = gr.inputs.Radio(['SACPP', 'PACPP', 'Benchamas', 'NPCPP'], label="Select CPP")
level_chkbox = gr.inputs.Checkbox(label="Level?")
tags_main_picker = gr.inputs.CheckboxGroup(main_category, label="Category")
granger_picker = gr.inputs.Radio(['Clustering (Indented Tree)', 'Granger Casuality Test'],label='Method')
granger_txt = gr.inputs.Textbox(lines=1, label='First Out Tag (Required for Granger Casuality Test Only) Example: CEN_LIC2720_PV Liquid Level')

gr.Interface(
    autoAD_clustering, 
    [
        start_date, 
        end_date, 
        frequency_method,
        selected_CPP,
        level_chkbox,
        tags_main_picker,
        granger_picker,
        granger_txt
    ], 
    #gr.outputs.Image(plot=True, label="clustering"), title="AUTO-AD",
    gr.outputs.Textbox(label="Result"),
    allow_flagging = False).launch(share = True)

Running locally at: http://127.0.0.1:7864/
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://34626.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7864/',
 'https://34626.gradio.app')

In [10]:
'''
import gradio as gr

def tranform_json_HTML():
  return "Place JSON to HTML function here"

jsonFilename = gr.inputs.Textbox(lines=1, 
                                 default=" Please fill in .json file name ", 
                                 label="JSON File"
                                )

iface = gr.Interface(
        tranform_json_HTML, 
        inputs=jsonFilename,
        outputs="text",
        layout="vertical",
        allow_screenshot = False,
        allow_flagging = False,
        )

iface.launch(share = True)
'''

'\nimport gradio as gr\n\ndef tranform_json_HTML():\n  return "Place JSON to HTML function here"\n\njsonFilename = gr.inputs.Textbox(lines=1, \n                                 default=" Please fill in .json file name ", \n                                 label="JSON File"\n                                )\n\niface = gr.Interface(\n        tranform_json_HTML, \n        inputs=jsonFilename,\n        outputs="text",\n        layout="vertical",\n        allow_screenshot = False,\n        allow_flagging = False,\n        )\n\niface.launch(share = True)\n'

In [9]:
'''
#Pull All Tags
results = spy.search({
    "Path": "Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes"
})
results2 = spy.search({
    "Path": "Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Sales Gas"
})
results3 = spy.search({
    "Path": "Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Skids >> SA-GGC2090-SACPP"
})
results4 = spy.search({
    "Path": "Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Skids >> SA-GGC2320-SACPP"
})
results5 = spy.search({
    "Path": "Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Streams"
})
print([results.shape, results2.shape, results3.shape, results4.shape, results5.shape])
# Print the output to the Jupyter page
results = results.merge(results2, how = 'outer')
results = results.merge(results3, how = 'outer')
results = results.merge(results4, how = 'outer')
results = results.merge(results5, how = 'outer')

print(results.shape)
results = results[results['Type']=='StoredSignal']
print(results.shape)
results
'''

,Path,Time,Count,Result
0,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Streams,00:00:04.36,160,Success


[(6237, 9), (2028, 9), (310, 9), (310, 9), (160, 9)]
(9045, 9)
(2873, 9)


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
7,00495A9E-1739-4D30-8399-D215E3FFCBC9,Chevron_AF_2017 >> Gulf of Thailand >> Satun >...,SACPP E-2530A,Cleaning FBE/Cleaning Startdate,NaN,StoredSignal,ns,UPSTREAM.ASBUTHAILAND.AF.GMWCPAPPV01119,False
10,00819725-7F6B-4B0E-8415-21836D5AD038,Chevron_AF_2017 >> Gulf of Thailand >> Satun >...,Motor 2,Vibration 1,NaN,StoredSignal,IPS,UPSTREAM.ASBUTHAILAND.AF.GMWCPAPPV01119,False
13,00A38B40-DDD2-4CD8-9C1C-40495EF42D5A,Chevron_AF_2017 >> Gulf of Thailand >> Satun >...,Motor 2,Vibration 1,NaN,StoredSignal,IPS,UPSTREAM.ASBUTHAILAND.AF.GMWCPAPPV01119,False
18,010D058A-E345-4A27-BAC9-5146CEBD4929,Chevron_AF_2017 >> Gulf of Thailand >> Satun >...,Motor 2,Vibration 3,NaN,StoredSignal,IPS,UPSTREAM.ASBUTHAILAND.AF.GMWCPAPPV01119,False
20,01301353-2E37-403B-BCE3-E180B176567D,Chevron_AF_2017 >> Gulf of Thailand >> Satun >...,SA-E2325_2327-SACPP,Outlet Temperature,NaN,StoredSignal,°F,UPSTREAM.ASBUTHAILAND.AF.GMWCPAPPV01119,False
...,...,...,...,...,...,...,...,...,...
9018,D8F2DFD0-871E-4326-AF50-340CAF43C45F,Chevron_AF_2017 >> Gulf of Thailand >> Satun >...,SACPP_FIT_2640_W,Gross Volume Water,NaN,StoredSignal,bbl,UPSTREAM.ASBUTHAILAND.AF.GMWCPAPPV01119,False
9025,DF0A33F0-2C75-4251-B7AA-E7427ECAD6F9,Chevron_AF_2017 >> Gulf of Thailand >> Satun >...,SACPP_FUEL_G,Outlet Temperature,NaN,StoredSignal,°F,UPSTREAM.ASBUTHAILAND.AF.GMWCPAPPV01119,False
9026,DF34491E-6F85-4BB6-A775-B38758D9362F,Chevron_AF_2017 >> Gulf of Thailand >> Satun >...,SACPP_FE-2511_C,Gross Volume Condy,NaN,StoredSignal,bbl,UPSTREAM.ASBUTHAILAND.AF.GMWCPAPPV01119,False
9039,F5A3AB52-CA6A-4389-8094-5BE84549291A,Chevron_AF_2017 >> Gulf of Thailand >> Satun >...,SACPP_FE-2510_G,Gross Volume Gas,NaN,StoredSignal,Mcf,UPSTREAM.ASBUTHAILAND.AF.GMWCPAPPV01119,False


In [10]:
'''
#Save to csv file
df = results.copy()
df = df[~df["Path"].str.contains('POSC')]
print(df["Path"][10])
df['ShownText'] = df['Path'].map(str) + '_' + df['Asset'].map(str) + '_' + df['Name'].map(str)
df['ShownText'] = df['ShownText'].str.slice(start = 39)
print(df["ShownText"][10])
df['Valve'] = [1 if x else 0 for x in df['ShownText'].str.contains("Valve")]
df['Level'] = [1 if x else 0 for x in df['ShownText'].str.contains("Level") & ~df['ShownText'].str.contains("Valve")]
df.head()
category = ['Coolers', '3 Phase Separators', 'Pumps', 'Glycol Contactors', '2 Phase Separators', 'Stabilizer Reboilers', 
            'Hydrocyclones', 'Filter Coalescers', 'Filters', 'Compressor-Expander Skids', 'WHRUs', 'Glycol Reboilers', 
            'Receivers', 'Launchers', 'Exchangers', 'Heaters', 'Glycol Flashs', 'IGFs', 'Stabilizers', 'Screw Compressors', 
            'Sales Gas Meters', 'Condensate Meters', 'Air Systems', 'Storage Tanks', 'Generator Skids', 'Sales Gas', 
            'Skids_SA-GGC2090-SACPP', 'Skids_SA-GGC2320-SACPP']
for cat in category:
    df.loc[df['ShownText'].str.contains(cat),'AssetTree'] = cat

df.to_csv('SACPP_Tags.csv')
'''

Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Coolers >> SA-E2285-SACPP
Satun >> Processes >> Coolers >> SA-E2285-SACPP_Motor 2_Vibration 1


In [11]:
#Test pulling data from Seeq
#df_test_pull = df[df['AssetTree']=="Hydrocyclones"]

In [12]:
#df_pull = spy.pull(df_test_pull.iloc[:,:9], start='2020-01-01', end='2020-01-07', grid = '30 min')

,ID,Path,Asset,Name,Count,Time,Result
108,04A35AB2-E412-4434-9664-56832CF9B1D5,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-V2065-SACPP,Inlet Pressure,289,00:00:00.03,Success
842,21DF8361-3F19-4DC8-8385-8EE05AEF2765,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-V2065-SACPP,Differential Pressure 1,289,00:00:00.08,Success
888,2387FA4D-5663-4041-89F2-F9C065EA8916,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-V2295-SACPP,Differential Pressure 1,289,00:00:00.09,Success
899,2445ABBB-4663-4191-BEB0-405604820898,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-V2065-SACPP,Water Outlet Pressure,289,00:00:00.11,Success
1329,3632EEDE-BB1D-4055-92AB-03AB7BA25756,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-DC2293_T2293-SACPP,Differential Pressure 1,289,00:00:00.06,Success
1657,43405F6F-E924-4699-BB4B-63395EF3BBF1,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-V2065-SACPP,Differential Pressure Control Valve,289,00:00:00.07,Success
1976,503EA496-8E30-4CB6-8169-70FC5353A0C7,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-DC2293_T2293-SACPP,Sand Outlet Pressure,289,00:00:00.11,Success
2264,5BC40A51-A645-45D2-A545-03EAB3F716EF,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-V2295-SACPP,Differential Pressure Control Valve,289,00:00:00.09,Success
2354,5F799C2A-0B50-4B99-9D1B-6CAA756D5E23,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-V2295-SACPP,Condensate Outlet Pressure,289,00:00:00.03,Success
2375,603F9E2F-5780-4C98-BE5A-A1A2B0CB2811,Chevron_AF_2017 >> Gulf of Thailand >> Satun >> Processes >> Hydrocyclones,SA-DC2063_T2063-SACPP,Inlet Pressure,289,00:00:00.03,Success
